## Import Data

In [1]:
import tensorflow as tf
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
import numpy as np

def to_one_hot(y, num_class=10):
  master = []
  for value in y[:,0]:
    row = np.zeros(num_class)
    row[value] = 1
    master.append(row)
  return np.array(master)

y_train_vec = to_one_hot(y_train)
y_test_vec = to_one_hot(y_test)

print('Shape of y_train_vec: ' + str(y_train_vec.shape))
print('Shape of y_test_vec: ' + str(y_test_vec.shape))

Shape of y_train_vec: (50000, 10)
Shape of y_test_vec: (10000, 10)


In [ ]:
import numpy 
rand_indices = numpy.random.permutation(50000)
train_indices = rand_indices[0:40000]
valid_indices = rand_indices[40000:50000]

x_val = x_train[valid_indices, :]
y_val = y_train_vec[valid_indices, :]

x_tr = x_train[train_indices, :]
y_tr = y_train_vec[train_indices, :]

print('Shape of x_tr: ' + str(x_tr.shape))
print('Shape of y_tr: ' + str(y_tr.shape))
print('Shape of x_val: ' + str(x_val.shape))
print('Shape of y_val: ' + str(y_val.shape))

Shape of x_tr: (40000, 32, 32, 3)
Shape of y_tr: (40000, 10)
Shape of x_val: (10000, 32, 32, 3)
Shape of y_val: (10000, 10)


## Noisy Data

In [ ]:
import keras 
train_datagen = keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2)

train_datagen.fit(x_tr)

val_datagen = keras.preprocessing.image.ImageDataGenerator()

val_datagen.fit(x_val)

# Making our Models

## Building ResNet

In [ ]:
from keras.layers import Input, Conv2D, ReLU, BatchNormalization,Add, AveragePooling2D, Flatten, Dense

#relu based batch normalization
#given a tensor x, we apply relu on it and use Batch Normalization on it
def relu_bn(inputs):
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

# a skip block with 2 convulotional layers
#size is the size of the layer
#Note that stride > 1 is downsampling. So we must downsample the original tensor as well to be able to add it to the downsampled one
def skip_block(x, filters, stride = 1, size = 3):
  y = Conv2D(kernel_size=size,
              strides= stride,
              filters=filters,
              padding="same")(x)
  y = relu_bn(y)
  y = Conv2D(kernel_size=size,
              strides=1,
              filters=filters,
              padding="same")(y)

  #downsample the original tensor
  #note that we could in theory just use max pooling or something here
  if stride != 1:
      x = Conv2D(kernel_size=1,
                  strides=stride,
                  filters=filters,
                  padding="same")(x)
  out = Add()([x, y])
  out = relu_bn(out)
  return out

from keras.models import Model
from keras import optimizers


#creates a Resnet model
#takes as input number of skip connections and when to downsample
#the default parameters are taken from the slides
def ResNet(ns = [3,5,5,3], initial_filters = 64, apool = 4):
  inputs = Input(shape = (32,32,3))

  current_filters = initial_filters

  t = BatchNormalization()(inputs)

  t = Conv2D(kernel_size=3, strides=1, filters=current_filters, padding="same")(t)
  t = relu_bn(t)

  for cnt, block in enumerate(ns):
    #Downscale if starting a new block unless it is the first block
    if cnt != 0:
      current_filters *= 2
      t = skip_block(t, current_filters, 2, 3)
    else:
      t = skip_block(t, current_filters, 1, 3)
    #make skip connections
    for connection in range(block-1):
      t = skip_block(t, current_filters, 1, 3)
    
  #Average pool and flatten followed by a dense layer
  t = AveragePooling2D(apool)(t)
  t = Flatten()(t)
  outputs = Dense(10, activation='softmax')(t)
  
  model = Model(inputs, outputs)

  return model

In [ ]:
modelRN = ResNet()
learning_rate = 1E-4
modelRN.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

In [ ]:
tf.keras.models.save_model(modelRN, "/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/RN_arch.h5")

In [ ]:
modelRN.summary()

## Basic 1-hidden layer neural network

code taken from official tensorflow documentation: https://www.tensorflow.org/tutorials/generative/cvae 

In [ ]:
from keras.layers import Dense, Input, Flatten
from keras import models

input_feat = Input(shape=(32,32,3,))

f1 = Flatten()(input_feat)
hidden1 = Dense(500, activation='relu')(f1)
hidden2 = Dense(250, activation='relu')(hidden1)
output = Dense(10, activation='softmax')(hidden2)

classifier = models.Model(input_feat, output)

classifier.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               1536500   
_________________________________________________________________
dense_5 (Dense)              (None, 250)               125250    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                2510      
Total params: 1,664,260
Trainable params: 1,664,260
Non-trainable params: 0
_________________________________________________________________


# Training Models

In [ ]:
modelRN = ResNet()
learning_rate = 1E-4
modelRN.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])
history = modelRN.fit(x_tr,y_tr, epochs = 6, validation_split=0.2)

Epoch 1/6
1000/1000 [==============================] - 56s 50ms/step - loss: 1.9174 - acc: 0.2975 - val_loss: 1.7991 - val_acc: 0.4305
Epoch 2/6
1000/1000 [==============================] - 49s 49ms/step - loss: 1.3133 - acc: 0.5293 - val_loss: 1.0958 - val_acc: 0.6209
Epoch 3/6
1000/1000 [==============================] - 49s 49ms/step - loss: 0.9269 - acc: 0.6745 - val_loss: 0.9776 - val_acc: 0.6710
Epoch 4/6
1000/1000 [==============================] - 49s 49ms/step - loss: 0.6652 - acc: 0.7695 - val_loss: 0.8496 - val_acc: 0.7085
Epoch 5/6
1000/1000 [==============================] - 50s 50ms/step - loss: 0.4495 - acc: 0.8452 - val_loss: 0.8285 - val_acc: 0.7320
Epoch 6/6
1000/1000 [==============================] - 49s 49ms/step - loss: 0.2939 - acc: 0.8977 - val_loss: 0.8812 - val_acc: 0.7415


In [ ]:
modelRN.save_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/ResNet.h5")

In [ ]:
modelRN = ResNet()
learning_rate = 1E-4
modelRN.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

history = modelRN.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), epochs=30, validation_data=val_datagen.flow(x_val, y_val, batch_size=32), validation_steps= 50)

Epoch 1/30
1250/1250 [==============================] - 69s 50ms/step - loss: 2.0193 - acc: 0.2559 - val_loss: 1.6724 - val_acc: 0.4281
Epoch 2/30
1250/1250 [==============================] - 62s 50ms/step - loss: 1.5902 - acc: 0.4244 - val_loss: 1.3028 - val_acc: 0.5450
Epoch 3/30
1250/1250 [==============================] - 62s 50ms/step - loss: 1.3692 - acc: 0.5151 - val_loss: 1.2218 - val_acc: 0.5881
Epoch 4/30
1250/1250 [==============================] - 62s 50ms/step - loss: 1.2061 - acc: 0.5745 - val_loss: 0.9453 - val_acc: 0.6787
Epoch 5/30
1250/1250 [==============================] - 62s 50ms/step - loss: 1.0998 - acc: 0.6158 - val_loss: 0.8890 - val_acc: 0.7013
Epoch 6/30
1250/1250 [==============================] - 62s 50ms/step - loss: 1.0037 - acc: 0.6496 - val_loss: 0.8449 - val_acc: 0.7138
Epoch 7/30
1250/1250 [==============================] - 62s 50ms/step - loss: 0.9342 - acc: 0.6769 - val_loss: 0.7786 - val_acc: 0.7294
Epoch 8/30
1250/1250 [==========================

In [ ]:
modelRN.save_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/ResNet_Noisy.h5")

In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet_Noisy.h5", monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=True, mode='auto', save_freq='epoch',
    options=None,
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_acc', min_delta=0, patience=6, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

In [ ]:
modelRN_deep = ResNet([3,7,7,3], 128)
learning_rate = 1E-4
modelRN_deep.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])
tf.keras.models.save_model(modelRN, "/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepRN_arch.h5")

In [ ]:


modelRN_deep = ResNet([3,7,7,3], 128)

modelRN_deep.load_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet_Noisy.h5")

learning_rate = 1E-4
modelRN_deep.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])



history = modelRN_deep.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), callbacks=[cp,es], epochs=100, validation_data=val_datagen.flow(x_val, y_val, batch_size=32), validation_steps= 50)

Epoch 1/100
1250/1250 [==============================] - 205s 145ms/step - loss: 0.4978 - acc: 0.8328 - val_loss: 0.4362 - val_acc: 0.8525
Epoch 2/100
1250/1250 [==============================] - 181s 145ms/step - loss: 0.4828 - acc: 0.8378 - val_loss: 0.3695 - val_acc: 0.8706
Epoch 3/100
1250/1250 [==============================] - 181s 145ms/step - loss: 0.4691 - acc: 0.8427 - val_loss: 0.4215 - val_acc: 0.8731
Epoch 4/100
1250/1250 [==============================] - 181s 145ms/step - loss: 0.4434 - acc: 0.8502 - val_loss: 0.7324 - val_acc: 0.8863
Epoch 5/100
1250/1250 [==============================] - 181s 145ms/step - loss: 0.4356 - acc: 0.8562 - val_loss: 0.5459 - val_acc: 0.8725
Epoch 6/100
1250/1250 [==============================] - 181s 145ms/step - loss: 0.4200 - acc: 0.8587 - val_loss: 5.0577 - val_acc: 0.8500
Epoch 7/100
1250/1250 [==============================] - 183s 146ms/step - loss: 0.4079 - acc: 0.8651 - val_loss: 0.8108 - val_acc: 0.8731
Epoch 8/100
1250/1250 [====

In [ ]:
modelRN_deep.save_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet_Noisy.h5")

In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet.h5", monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=True, mode='auto', save_freq='epoch',
    options=None,
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_acc', min_delta=0, patience=6, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

modelRN_deep = ResNet([3,7,7,3], 128)
learning_rate = 1E-4
modelRN_deep.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])
history = modelRN_deep.fit(x_tr,y_tr, epochs = 100, callbacks=[cp,es], validation_data=(x_val,y_val))

Epoch 1/100
1250/1250 [==============================] - 121s 91ms/step - loss: 1.9451 - acc: 0.3202 - val_loss: 2.4660 - val_acc: 0.5572
Epoch 2/100
1250/1250 [==============================] - 113s 90ms/step - loss: 1.1193 - acc: 0.6063 - val_loss: 5.0668 - val_acc: 0.6479
Epoch 3/100
1250/1250 [==============================] - 113s 90ms/step - loss: 0.7814 - acc: 0.7317 - val_loss: 0.7911 - val_acc: 0.7296
Epoch 4/100
1250/1250 [==============================] - 112s 90ms/step - loss: 0.5900 - acc: 0.7963 - val_loss: 0.7815 - val_acc: 0.7703
Epoch 5/100
1250/1250 [==============================] - 112s 90ms/step - loss: 0.4625 - acc: 0.8425 - val_loss: 0.9422 - val_acc: 0.7742
Epoch 6/100
1250/1250 [==============================] - 112s 90ms/step - loss: 0.3627 - acc: 0.8756 - val_loss: 0.6450 - val_acc: 0.8065
Epoch 7/100
1250/1250 [==============================] - 112s 90ms/step - loss: 0.2760 - acc: 0.9062 - val_loss: 0.6399 - val_acc: 0.8104
Epoch 8/100
1250/1250 [===========

In [ ]:
modelRN_deep.save_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet.h5")

Evaluation on test set

In [ ]:
modelRN.load_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/ResNet.h5")
modelRN.evaluate(x_test, y_test_vec)

313/313 [==============================] - 5s 15ms/step - loss: 0.9113 - acc: 0.7387


[0.9112823009490967, 0.7386999726295471]

In [ ]:
modelRN.load_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/ResNet_Noisy.h5")
modelRN.evaluate(x_test, y_test_vec)

313/313 [==============================] - 5s 15ms/step - loss: 0.3468 - acc: 0.8844


[0.34679803252220154, 0.8844000101089478]

In [ ]:
modelRN_deep = ResNet([3,7,7,3], 128)

learning_rate = 1E-4
modelRN_deep.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=learning_rate),
              metrics=['acc'])

modelRN_deep.load_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet_Noisy.h5")
modelRN_deep.evaluate(x_test, y_test_vec)

313/313 [==============================] - 30s 42ms/step - loss: 0.5109 - acc: 0.8572


[0.4884413182735443, 0.8567000031471252]

In [ ]:
tf.keras.models.save_model(modelRN_deep, "/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepRN_arch.h5")

In [ ]:
modelRN_deep = tf.keras.models.load_model("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepRN_arch.h5")

modelRN_deep.load_weights("/content/drive/MyDrive/Trick_NN/Models/CIFAR_Models/DeepResNet.h5")
modelRN_deep.evaluate(x_test, y_test_vec)

313/313 [==============================] - 14s 42ms/step - loss: 0.8313 - acc: 0.8245


[0.831285834312439, 0.8245000243186951]